<a href="https://colab.research.google.com/github/ravi-prakash1907/A-tracking-of-COVID-19/blob/master/bestNFPSelector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

In [1]:
### libraries
#from PIL import Image,ImageOps
#import os
#import math
#from statistics import mean
#
#import pandas as pd
#import numpy as np
#import pickle


### for data processing
import pandas as pd
#import numpy as np
#import os
#from PIL import Image,ImageOps
#
### over sampling for training data balancing
from math import sqrt
#from matplotlib import pyplot
#from numpy import where
#
### for getting data from g-drive
from google.colab import drive
#
## Other Purpose
#import warnings
#warnings.filterwarnings('ignore')
## own
from configration import *

In [4]:
## GLOBALS
DRIVE_MOUNT_SRC = None
IMG_DIR_LOC = None

g_setter = setGlobals('config.json') 
GLOBALS = g_setter.setGlobalVars()

## ML Model Based Classification

### Feature extractor class

In [ ]:
# take all necessary features

### FP Image & Model Loader and Predictor

In [20]:
# get data:  from drive loc

In [21]:
# models loaded

In [22]:
# export prediction + feature dataset

## Choose Best N Pics

In [5]:
class fpPrepare:
  def __init__(self):
    self.INT_NORM_FACTOR = int(sqrt(GLOBALS['DEC_NORM_FACTOR']))
    pass
  
  ## sample collector
  def getSamples(self, df: pd.core.frame.DataFrame, picLbl: int, sCount: int=GLOBALS['SAMPLE_COUNT']):
    gotdf = df.copy()
    ## normalize attr
    COLS_TO_NORMALIZE = list(set(df.columns) - set(GLOBALS['COLS_NOT_TO_NORMALIZE']))
    for col in COLS_TO_NORMALIZE:
      sample = gotdf[col][GLOBALS['INIT_INDEX']]
      if int(sample):
        gotdf[col] = round(gotdf[col]//self.INT_NORM_FACTOR)*self.INT_NORM_FACTOR
      else:
        gotdf[col] = round(gotdf[col]*GLOBALS['DEC_NORM_FACTOR'])/GLOBALS['DEC_NORM_FACTOR']
    
    gotdf = gotdf.sort_values(by = GLOBALS['COL_PRIORITY'])
    begIndex = GLOBALS['INIT_INDEX']
    
    reqImagePool = gotdf[gotdf[GLOBALS['LBL_COL']] == picLbl]
    reqImgCount = reqImagePool.shape[GLOBALS['INIT_INDEX']]
    sCount = sCount if sCount < reqImgCount else reqImgCount-1

    sampleDFs = reqImagePool[begIndex:begIndex+sCount]
    reqImages = sampleDFs
    return reqImages


  ## common part extractor
  def fetchCommon(self, pxlImgList: list):
    fpShape = pxlImgList[GLOBALS['INIT_INDEX']].shape
    pxlPosTracker = np.zeros(shape=fpShape, dtype=GLOBALS['DEFAULT_ENCODING']) 

    rows, cols = fpShape
    totalPix = len(pxlImgList)
    for r in range(rows):
      for c in range(cols):
        thisPosVals = [pxlImgList[i][r][c] 
                          for i in range(totalPix)]
        ## bring normalizing at scale of self.INT_NORM_FACTOR by default
        thisPosVals = [px//self.INT_NORM_FACTOR*self.INT_NORM_FACTOR for px in thisPosVals]
        thisPosVals = set(thisPosVals)
        if len(thisPosVals) > GLOBALS['TRUE']:
          continue
        else:
          pxlPosTracker[r][c] = GLOBALS['TRUE']
        
    return pxlPosTracker

In [ ]:
LABELLED_DATA_LOC
DEC_NORM_FACTOR
SAMPLE_COUNT
COLS_NOT_TO_NORMALIZE
INIT_INDEX
DROP_COLS

COL_PRIORITY
INIT_INDEX
DEFAULT_ENCODING
TRUE
BEST_LBL

## Application

In [6]:
## init obj
thisPrep = fpPrepare()

In [9]:
dataLoc = GLOBALS['LABELLED_DATA_LOC']
## load labelled dataset
labelledFP = pd.read_csv(dataLoc)
labelledFP.drop(columns=GLOBALS['DROP_COLS'], inplace=True)

In [12]:
# getting best n inmages
reqImg = thisPrep.getSamples(labelledFP, picLbl=GLOBALS['BEST_LBL'])
reqImg 

,image_name,mean,variance,avg_bdd,avg_rtovratio,SqSum_RVRatio,avg_Orientation,label
226,00009_48.bmp,110.0,9910.0,1100,0.40,80,220.0,1
219,00009_16.bmp,120.0,10360.0,1030,0.42,90,190.0,1
100,00004_19.bmp,130.0,10980.0,1540,0.29,100,200.0,1
65,00002_46.bmp,130.0,11080.0,1430,0.28,100,190.0,1
64,00002_44.bmp,130.0,11100.0,1280,0.28,100,200.0,1
104,00004_36.bmp,130.0,11110.0,1160,0.33,100,190.0,1
34,00001_05.bmp,130.0,11170.0,1010,0.39,140,190.0,1
33,00001_04.bmp,130.0,11240.0,1000,0.33,100,190.0,1
61,00002_24.bmp,130.0,11460.0,1540,0.18,50,190.0,1
50,00001_67.bmp,130.0,11580.0,1250,0.20,70,200.0,1
